In [ ]:
!pip install requests
!pip install pandas

In [ ]:
import requests
import pandas as pd
from io import StringIO

url = 'https://storage.googleapis.com/the_public_bucket/wine-clustering.csv'  # Replace with the actual API endpoint
response = requests.get(url)

if response.status_code == 200:
    data = response.text
    df = pd.read_csv(StringIO(data))
else:
    print('Failed to retrieve data from the API')

In [ ]:
print(df.describe())

# Scatter plots
sns.pairplot(df[['Alcohol', 'Malic_Acid', 'Flavanoids', 'Nonflavanoid_Phenols', 'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']])
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Wine Heatmap')
plt.show()

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df)

#DataFrame with the scaled features
df_scaled = pd.DataFrame(scaled_features, columns=df.columns)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

silhouette_scores = []

# Calculate silhouette score 
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(df_scaled)
    silhouette = silhouette_score(df_scaled, cluster_labels)
    silhouette_scores.append(silhouette)

# Plot 
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs. Number of Clusters')
plt.show()

In [ ]:

import numpy as np

clustering = KMeans(n_clusters=3, max_iter = 300)
clustering.fit(df_scaled)


In [ ]:
df['Kmeans_clusters']=clustering.labels_
df.head(10)

In [ ]:
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=2) 
pca = pca.fit_transform(df_scaled)

# Create a new DataFrame with the principal components
pca_df = pd.DataFrame(data=pca, columns=['Component_1', 'Component_2'])

# Concatenate 
pca_wine = pd.concat([pca_df, df[['Kmeans_clusters']]], axis=1)

In [ ]:
pca_wine

In [ ]:
fig = plt.figure(figsize = (6,6))

ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Component 1', fontsize = 15)
ax.set_ylabel('Component 2', fontsize = 15)
ax.set_title('Principal Components', fontsize = 20)

color_theme = np.array(['blue', 'green', 'orange'])
ax.scatter(x = pca_wine.Component_1, y= pca_wine.Component_2,c= color_theme[pca_wine.Kmeans_clusters],s=50)

plt.show()

In [ ]:

#mean values
cluster_means = df.groupby('Kmeans_clusters').mean()

# Compare mean values
cluster_means

In [ ]:
correlation = df['Total_Phenols'].corr(df['Flavanoids'])
print(correlation)